# Config & App Settings Tour

This notebook shows how to inspect the default `ConfigManager`, load a custom YAML, and bootstrap `AppConfig` so all RAGdoll helpers share the same settings and helpers.


In [11]:
from pathlib import Path
import json
import os

from ragdoll.config import Config
from ragdoll.app_config import bootstrap_app

custom_config_path = Path('examples/app_config_demo.yaml').resolve()
print('Custom config path:', custom_config_path)


Custom config path: C:\dev\RAGdoll\examples\examples\app_config_demo.yaml


## Inspecting the Default Configuration

Use `ConfigManager` with no arguments to see the shipped defaults and what each typed section exposes.


In [12]:
default_config = Config()
print('Cache TTL (default):', default_config.cache_config.cache_ttl)
print('Ingestion batch size:', default_config.ingestion_config.batch_size)
print('Monitor enabled:', default_config.monitor_config.enabled)
print('Prompt templates keys:', list(default_config.llm_prompts_config.model_dump().keys()))
loader_map = default_config.get_loader_mapping()
print('Default loader count:', len(loader_map))
print('Sample loader map entries:', json.dumps(list(loader_map.items())[:5], indent=2))


Cache TTL (default): 3600
Ingestion batch size: 100
Monitor enabled: False
Prompt templates keys: ['entity_extraction', 'extract_relationships', 'coreference_resolution', 'entity_relationship_gleaning', 'entity_relationship_gleaning_continue']
Default loader count: 21
Sample loader map entries: [
  [
    ".json",
    "langchain_community.document_loaders.JSONLoader"
  ],
  [
    ".jsonl",
    "langchain_community.document_loaders.JSONLoader"
  ],
  [
    ".yaml",
    "langchain_community.document_loaders.JSONLoader"
  ],
  [
    ".csv",
    "langchain_community.document_loaders.CSVLoader"
  ],
  [
    ".epub",
    "langchain_markitdown.loaders.EpubLoader"
  ]
]


## Load a Custom YAML and Inspect Overrides

Create a bespoke YAML and feed it to `ConfigManager` to inspect the sections that matter to your app.


In [13]:
custom_config = Config(str(custom_config_path))
print('Custom cache TTL:', custom_config.cache_config.cache_ttl)
print('Custom ingestion loaders mapping:', json.dumps(custom_config.get_loader_mapping(), indent=2))
print('Vector store params (coerced):', custom_config.vector_store_config.params)
print('Entity extraction hint:', custom_config.entity_extraction_config.llm_provider_hint)


No file extension loaders were registered.


Custom cache TTL: 86400
Custom ingestion loaders mapping: {}
Vector store params (coerced): {}
Entity extraction hint: None


## Bootstrapping AppConfig, Overrides, and Shared Helpers

`AppConfig` unifies the validated `ConfigManager` with helpers such as `CacheManager`, `MetricsManager`, and prompt templates. Override blocks let you tweak sections without editing the YAML.


In [14]:
os.environ['RAGDOLL_CONFIG_PATH'] = str(custom_config_path)
base_app = bootstrap_app()
print('Loaded via env var - cache TTL:', base_app.config.cache_config.cache_ttl)
overrides = {
    'cache': {'cache_ttl': 10},
    'ingestion': {'batch_size': 1},
}
override_app = bootstrap_app(config_path=str(custom_config_path), overrides=overrides)
print('Override cache TTL:', override_app.config.cache_config.cache_ttl)
print('Override ingestion batch:', override_app.config.ingestion_config.batch_size)
public_prompts = override_app.get_prompt_templates()
print('Cached prompt templates sample:', list(public_prompts.keys())[:3])
cache_manager = override_app.get_cache_manager()
print('Shared cache directory:', cache_manager.cache_dir)
print('Cache TTL from manager:', cache_manager.ttl_seconds)


Loaded via env var - cache TTL: 86400
Override cache TTL: 10
Override ingestion batch: 1
Cached prompt templates sample: ['entity_extraction', 'extract_relationships', 'coreference_resolution']
Shared cache directory: C:\Users\nsast\.ragdoll\cache
Cache TTL from manager: 10


## Instantiating Ragdoll with `AppConfig`

Now create a `Ragdoll` instance using the shared `override_app`. For this notebook we use `FakeEmbeddings`, which keeps the vector store creation local while still exercising how every component shares the validated config.


In [15]:
from ragdoll.ragdoll import Ragdoll
from ragdoll.embeddings import FakeEmbeddings

fake_embeddings = FakeEmbeddings(size=16)
ragdoll = Ragdoll(app_config=override_app, embedding_model=fake_embeddings)
print('Loader batch size (shared config):', ragdoll.ingestion_service.batch_size)
print('Vector store type:', ragdoll.config_manager.vector_store_config.store_type)
print('Vector store params:', ragdoll.config_manager.vector_store_config.params)
print('Cache TTL via ragdoll config:', ragdoll.app_config.config.cache_config.cache_ttl)
print('Loader map size:', len(ragdoll.config_manager.get_loader_mapping()))


No file extension loaders were registered.
No model specified and no default model found in config.
No file extension loaders were registered.


Loader batch size (shared config): 1
Vector store type: chroma
Vector store params: {}
Cache TTL via ragdoll config: 10
Loader map size: 0


### Sharing Helpers & Metrics

Demonstrate that modules continue to pull from the same `AppConfig` helpers: cache manager, metrics manager, and prompt templates remain in lockstep.


In [16]:
metrics_manager = ragdoll.app_config.get_metrics_manager()
print('Metrics directory:', metrics_manager.metrics_dir)
print('Metrics capture enabled:', ragdoll.app_config.config.monitor_config.collect_metrics)
prompts = ragdoll.app_config.get_prompt_templates()
print('Sample prompt template keys:', list(prompts.keys())[:5])


Metrics directory: C:\Users\nsast\.ragdoll\metrics
Metrics capture enabled: True
Sample prompt template keys: ['entity_extraction', 'extract_relationships', 'coreference_resolution', 'entity_relationship_gleaning', 'entity_relationship_gleaning_continue']


### What You Just Saw

1. The config module exposes typed sections that you can inspect programmatically.
2. Pointing at a custom YAML lets you swap defaults with your own teams' settings.
3. `AppConfig` keeps helpers, cache/metrics managers, and prompt templates in one place; passing it into `Ragdoll` wires every module to the same settings.
